<a href="https://colab.research.google.com/github/ar851060/palm_detection_prediction/blob/main/CCBDA_final_SimCLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torchvision.models import resnet18
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder, DatasetFolder
import numpy as np
from torch.utils.data import DataLoader, Subset, Dataset
import os
from tqdm.auto import tqdm
from PIL import Image

In [ ]:
# !unzip -q /content/drive/MyDrive/hw2.zip

In [ ]:
!nvidia-smi

Wed Jun 22 16:03:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
def xt_xent(u, v, temperature=0.5):
    N = u.shape[0]
    z = torch.cat([u, v], dim=0)
    z = F.normalize(z, p=2, dim=1) # L1/L2 normalization (Lp)
    s = torch.matmul(z, z.t()) / temperature # matrix product
    mask = torch.eye(2 * N).bool().to(z.device) # eye: diagonal matrix
    s = torch.masked_fill(s, mask, -float('inf')) # 
    label = torch.cat([
        torch.arange(N, 2 * N),
        torch.arange(N)]).to(z.device)

    loss = F.cross_entropy(s, label)
    return loss

In [ ]:
# def KNN(emb, cls, batch_size, Ks=[1, 10, 50, 100]):
#     """Apply KNN for different K and return the maximum acc"""
#     preds = []
#     mask = torch.eye(batch_size).bool().to(emb.device)
#     mask = F.pad(mask, (0, len(emb) - batch_size))
#     for batch_x in torch.split(emb, batch_size):
#         dist = torch.norm(
#             batch_x.unsqueeze(1) - emb.unsqueeze(0), dim=2, p="fro")
#         now_batch_size = len(batch_x)
#         mask = mask[:now_batch_size]
#         dist = torch.masked_fill(dist, mask, float('inf'))
#         # update mask
#         mask = F.pad(mask[:, :-now_batch_size], (now_batch_size, 0))
#         pred = []
#         for K in Ks:
#             knn = dist.topk(K, dim=1, largest=False).indices
#             knn = cls[knn].cpu()
#             pred.append(torch.mode(knn).values)
#         pred = torch.stack(pred, dim=0)
#         preds.append(pred)
#     preds = torch.cat(preds, dim=1)
#     accs = [(pred == cls.cpu()).float().mean().item() for pred in preds]
#     return max(accs)

In [ ]:
transform = transforms.RandomOrder([
    # transforms.RandomResizedCrop((96,96), scale=(0.2, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=96//20*2+1, sigma=(0.1, 2.0))], p=0.5),
    # transforms.RandomErasing(p=0.5),
    transforms.RandomApply([transforms.RandomAffine(degrees = 45)],p=0.5),
    transforms.RandomApply([transforms.RandomPerspective()],p=0.2),
])

In [ ]:
import re
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self,path,transform = transform, files = None):
        super(ImageDataset).__init__()
        self.path = path
        self.files = sorted_alphanumeric([os.path.join(path,x) for x in os.listdir(path)])
        if files != None:
            self.files = files
        # print(self.files)
        self.transform = transform
        self.tensor = transforms.ToTensor()
        # self.norm = transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.tensor(im)
        im = torch.cat([im, im, im], dim=0)
        im1 = self.transform(im)
        im2 = self.transform(im)
        # im = self.norm(im)
        # im1 = self.norm(im1)
        # im2 = self.norm(im2)
        return im, im1, im2


In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.in_dim = 256
        self.out_dim = 1000
        self.encoder = resnet18(pretrained = True, num_classes = self.out_dim)
        self.num_features = self.encoder.fc.out_features
        self.encoder.fc = nn.Identity()
        self.fc = nn.Sequential(
            nn.Linear(512,128, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace = True),
            nn.Linear(128,32, bias=False),
            nn.BatchNorm1d(32),
        )
    def forward(self,x):
        h = self.encoder(x)
        z = self.fc(h)
        return h, z

In [ ]:
batch_size = 256
# dir = "./hw2"
unlabeled_set = ImageDataset("/content/drive/Shareddrives/CCBDA_final/allLines")
unlabeled_loader = DataLoader(unlabeled_set, batch_size = batch_size, shuffle = True)
# test_tfm = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))]) 
# test_set = DatasetFolder("/content/hw2/test", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
final_set = ImageDataset("/content/drive/Shareddrives/CCBDA_final/allLines")
final_loader = DataLoader(final_set, batch_size = batch_size, shuffle = False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4 ,weight_decay = 1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = len(unlabeled_loader))
best_score = 1000
n_epochs = 200
# model.load_state_dict(torch.load('/content/drive/MyDrive/model.ckpt'))
for epoch in range(n_epochs):
    model.train()
    train_loss = []

    for _,xi,xj in tqdm(unlabeled_loader):
        _, zi = model(xi.to(device))
        _, zj = model(xj.to(device))
        loss = xt_xent(zi, zj, 0.1) 
        loss.backward()
        train_loss.append(loss)
        optimizer.step()
        optimizer.zero_grad()
        
    if epoch > 9:
        scheduler.step()
    train_loss = sum(train_loss) / len(train_loss)
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}")

    # model.eval()
    # embedding = []
    # classes = []
    # for batch in test_loader:
    #     imgs, labels = batch
    #     with torch.no_grad():
    #         h, _ = model(imgs.to(device))
    #         embedding.append(h)
    #         classes.append(labels)
    # embedding = torch.cat(embedding, dim = 0)
    # classes = torch.cat(classes, dim = 0)
    # # acc = KNN(embedding, classes, batch_size=16)
    # print(f"[ Test | {epoch + 1:03d}/{n_epochs:03d} ] Accuracy = {acc:.5f}")
    
    if best_score > train_loss:
        best_score = train_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/model.ckpt')

# print(best_score)
# embedding = []
# model = ResNet().to(device)
# model.load_state_dict(torch.load('/content/drive/MyDrive/model.ckpt'))
# model.eval()
# for x,_,_ in final_loader:
#     with torch.no_grad():
#         _, z = model(x.to(device))
#         embedding.append(z.detach())
# embedding = torch.cat(embedding, dim = 0)
# embedding = embedding.cpu().detach().numpy()
# np.save("/content/drive/MyDrive/embedding.npy", embedding)

  0%|          | 0/6 [00:00<?, ?it/s]

[ Train | 001/200 ] loss = 6.42883


  0%|          | 0/6 [00:00<?, ?it/s]

[ Train | 002/200 ] loss = 4.71803


  0%|          | 0/6 [00:00<?, ?it/s]

...

[ Train | 198/200 ] loss = 0.17390


  0%|          | 0/6 [00:00<?, ?it/s]

[ Train | 199/200 ] loss = 0.17446


  0%|          | 0/6 [00:00<?, ?it/s]

[ Train | 200/200 ] loss = 0.17137


In [ ]:
final_set = ImageDataset("/content/drive/Shareddrives/CCBDA_final/allLines")
final_loader = DataLoader(final_set, batch_size = batch_size, shuffle = False)

In [ ]:
embedding = []
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet().to(device)
model.load_state_dict(torch.load('/content/drive/Shareddrives/CCBDA_final/simclr.ckpt'))
model.eval()
for x,_,_ in final_loader:
    with torch.no_grad():
        _, z = model(x.to(device))
        embedding.append(z.detach())
embedding = torch.cat(embedding, dim = 0)
embedding = embedding.cpu().detach().numpy()
np.save("/content/drive/Shareddrives/CCBDA_final/embLine/embeddingLine.npy", embedding)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
embedding.shape

(1442, 32)

In [ ]:
import pandas as pd
myemb = np.load("/content/drive/MyDrive/embedding.npy")
myemb = torch.from_numpy(myemb).to(device)
ans = pd.read_csv("/content/unlabeled.txt", sep=" ", header=None)
ans = torch.from_numpy(ans.iloc[:,1].to_numpy()).to(device)

FileNotFoundError: ignored

In [ ]:
KNN(myemb,ans,batch_size=16)